# ResNet-50 Learning

Time to put our dataset to use! We'll use pytorch-lightning as a trainer!

If you'd like to see, how well does the backbone work compared to using transfer with the main article ResNet-50, see ResNet-50 Backbone Testing notebook.

### Constants and imports

In [17]:
import os
import torch
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from lightning.pytorch import Trainer, seed_everything, LightningModule
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torchvision.io import read_image
from torchvision.transforms import v2
from huggingface_hub import PyTorchModelHubMixin
from safetensors.torch import save_model
import json

METADATA_DIRECTORY = 'metadata'
USE_PREPROCESSED_IMAGES = True
DATASET_FILE = os.path.join(METADATA_DIRECTORY, 'dataset-preprocessed.csv') if USE_PREPROCESSED_IMAGES else os.path.join(METADATA_DIRECTORY, 'dataset.csv')
CHECKPOINTS_DIRECTORY = 'checkpoints'
LOAD_LAST_CHECKPOINT = False
TEST_SIZE = 0.3
RANDOM_STATE = 42
BATCH_SIZE = 32
EPOCH_COUNT = 90

### Preparing directories

In [2]:
if not os.path.exists(CHECKPOINTS_DIRECTORY):
    os.makedirs(CHECKPOINTS_DIRECTORY)

### Loading the dataset

In [3]:
df = pd.read_csv(DATASET_FILE)

classes = sorted(pd.unique(df['class']))

/tmp/ipykernel_111361/1652983266.py:1: DtypeWarning: Columns (4,5,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(DATASET_FILE)


In [4]:
len(classes)

45

In [5]:
len(df.index)

318563

### Splitting the dataset

In [6]:
X_train, X_test, y_train, y_test = train_test_split(df['path'], df['class'], test_size=TEST_SIZE, random_state=RANDOM_STATE)

### Define transformations

In [7]:
train_transforms = v2.Compose([
    v2.RandomHorizontalFlip(p=0.5),
    v2.RandomVerticalFlip(p=0.2),
    v2.RandomRotation(5),
    v2.Resize(size=(224, 224), antialias=True),
    v2.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.05)
])

test_transforms = v2.Compose([
    v2.Resize(size=(224, 224), antialias=True),
])

### Creating dataloaders

In [8]:
class MuseumCC0Dataset(Dataset):
    def __init__(self, Xs, ys, transform=None):
        self.Xs = list(Xs)
        self.ys = torch.tensor([classes.index(y) for y in ys])
        self.transform = transform
        
    def __len__(self):
        return len(self.Xs)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        image = read_image(self.Xs[idx])

        image = image.float() / 255

        if image.size()[0] == 1:
            image = image.repeat(3, 1, 1)

        if image.size()[0] == 4:
            image = image[1:4]
        
        if self.transform:
            image = self.transform(image)

        return image, self.ys[idx]

train_dataset = MuseumCC0Dataset(X_train, y_train, train_transforms)
test_dataset = MuseumCC0Dataset(X_test, y_test, test_transforms)

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=5)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, num_workers=5)

### Creating the trainer

In [18]:
class MuseumCC0Module(torch.nn.Module, PyTorchModelHubMixin):
    def __init__(self, classes_count):
        super().__init__()
        self.model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', weights=None)
        self.model.fc = torch.nn.Linear(self.model.fc.in_features, classes_count)

    def forward(self, data):
        return self.model(data)
                               
class MuseumCC0LightningModule(LightningModule, PyTorchModelHubMixin):
    def __init__(self, classes_count):
        super().__init__()

        self.model = MuseumCC0Module(classes_count)
        self.loss_module = torch.nn.CrossEntropyLoss()
        sd = self.state_dict()
        sd["metadata"] = {"format": "pt"}
        

    def forward(self, imgs):
        return self.model(imgs)

    def configure_optimizers(self):
        optimizer = torch.optim.SGD(self.model.parameters(), lr=0.1, weight_decay=1e-4)
        scheduler = lr_scheduler.LambdaLR(optimizer, lambda epoch: 0.1 ** (epoch // 30))
        return [optimizer], [scheduler]

    def training_step(self, batch, batch_idx):
        images, labels = batch

        preds = self.model(images)
        
        loss = self.loss_module(preds, labels)

        acc = (preds.argmax(dim=-1) == labels).float().mean()

        self.log("train_acc", acc, on_step=True, on_epoch=True)
        self.log("train_loss", loss)
        
        return loss 

    def validation_step(self, batch, batch_idx):
        images, labels = batch
        preds = self.model(images)

        loss = self.loss_module(preds, labels)

        acc = (labels == preds.argmax(dim=-1)).float().mean()

        self.log("val_acc", acc, on_epoch=True, prog_bar=True)
        self.log("val_loss", loss, on_epoch=True, prog_bar=True)

    def _save_pretrained(self, save_directory) -> None:
        model_to_save = self.module if hasattr(self, "module") else self 
        save_model(model_to_save, str(save_directory / "model.safetensors"), {"format": "pt"})

trainer = Trainer(default_root_dir=CHECKPOINTS_DIRECTORY, max_epochs=EPOCH_COUNT)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


### Fitting the model

In [19]:
def get_newest_model():
    logs_paths = os.path.join(CHECKPOINTS_DIRECTORY, 'lightning_logs')
    newest_version = -1
    checkpoint_path = None
    for dir in os.listdir(logs_paths):
        dir_path = os.path.join(logs_paths, dir)
        if os.path.isdir(dir_path) and 'version_' in dir:
            num = int(dir.replace('version_', ''))
            if newest_version > num:
                continue
            
            listing = os.listdir(dir_path)
            if 'checkpoints' in listing:
                check_dir = os.path.join(dir_path, 'checkpoints')
                for filename in os.listdir(check_dir):
                    if '.ckpt' in filename:
                        checkpoint_path = os.path.join(check_dir, filename)
                        newest_version = num

    print(f'Using checkpoint version {newest_version}!') 
    return checkpoint_path

In [20]:
torch.set_float32_matmul_precision('high')

config = { "classes_count": len(classes) }
model = MuseumCC0LightningModule(**config)

if not LOAD_LAST_CHECKPOINT:
    trainer.fit(model, train_dataloader, test_dataloader)
else:
    trainer.fit(model, train_dataloader, test_dataloader, ckpt_path=get_newest_model())



Using cache found in /home/macron/.cache/torch/hub/pytorch_vision_v0.10.0
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type             | Params
-------------------------------------------------
0 | model       | MuseumCC0Module  | 23.6 M
1 | loss_module | CrossEntropyLoss | 0     
-------------------------------------------------
23.6 M    Trainable params
0         Non-trainable params
23.6 M    Total params
94.401    Total estimated model params size (MB)


Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

Training: |                                               | 0/? [00:00<?, ?it/s]

### Uploading the model to the HuggingFace Repo

In [21]:
UPLOAD_TO_HUGGING_FACE = True

In [22]:
if UPLOAD_TO_HUGGING_FACE:
    with open('secrets.json') as f:
        data = json.load(f)
        HUGGING_FACE_TOKEN = data["hugging_face_token"]
    #model.model.save_pretrained("cc0-resnet", config=config)

    model.push_to_hub("Bulke/cc0-resnet", config=config, use_auth_token=HUGGING_FACE_TOKEN, private=True)

model.safetensors:   0%|          | 0.00/94.6M [00:00<?, ?B/s]

In [ ]:
model.model.from_pretrained("cc0-resnet")